# Atlântico Academy Future
### VISÃO GERAL
Uma empresa contratante deseja estabelecer termos de maior relevância em um documento específico. Neste caso, considere o histórico de exames, consultas e procedimentos realizados por um paciente. Um sistema deve ser desenvolvido para que o médico possa ter uma visão geral do histórico do paciente sem a necessidade de analisar documento por documento. Com
base nesta importância, vamos desenvolver uma etapa deste sistema. Tokenizar um texto,realizar remoção de stopwords, aplicar o processo de lematização e fazer uma análise quantitativa deste. Neste caso, vamos comparar duas estratégias , se possível. A primeira utilizando a lib stanza e a segunda uma análise com base em acesso a um dicionário.

# 1) Carregar o conjunto de documentos em PDF e armazená-los em alguma estrutura de dados

In [1]:
import PyPDF2
import os
import re

def read_pdf_file(file):
    pdf_file = open(file, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    page_content_all= ''
    for i in range(number_of_pages):
        page_content = read_pdf.getPage(i).extractText()
        page_content_all += page_content
    return page_content_all

In [2]:
def load_pfds(patch_files):
    os.chdir(patch_files)
    file_base_all = []
    for file in os.listdir():
        if file.endswith(".pdf"):
            file_base_all.append(read_pdf_file(os.path.join(patch_files, file)))
    return file_base_all

In [3]:
def join_and_remove_breaks(base):
    list_parsed = []
    for i in range(len(base)):
        list_parsed.append(re.sub('\n', '', base[i]))
    return list_parsed

In [4]:
#corpus = join_and_remove_breaks(load_pfds(r'C:\Users\r211315\Documents\ia_express\arquivos'))
corpus = join_and_remove_breaks(load_pfds(r'/home/vagnersv/my_tensorflow/arquivos'))

In [5]:
len(corpus)

2

In [6]:
corpus

['Uma TARTARUGA lenta pula sobre a raposa preguiçosa. QUE RAPASA DO BARALHO! ',
 'Uma rápida RAPOSA marrom pula sobre o CÃO preguiçoso. Que raposa marota! ']

# 2) Realizar o pré-processamento destes ( tokenização e remoção de stop words, deixar todos os caracteres minúsculos...)

Remoção de Palavras Vazias(e.g., artigos, preposições, etc.), que possuem alta frequência em todos os documentos, podem ser removidas da contagem para melhorar a distinção entre documentos

[CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [7]:
for i in range(len(corpus)):
    corpus[i]= corpus[i].lower()

In [9]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

def removestopwords(texto):
    frases = []
    for palavras in texto:
        semstop = [p for p in palavras.split() if p not in stopwords]
        frases.append(semstop)
    return frases

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vagnersv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
corpus_sem_stopwords = removestopwords(corpus)

In [15]:
def convert_list_to_string(corpus, seperator=' '):
    return  seperator.join(corpus)

In [16]:
corpus_sem_stopwords= list(map(convert_list_to_string, corpus_sem_stopwords))

In [ ]:
corpus_sem_stopwords

In [17]:
from nltk.stem import PorterStemmer
import string

#######
# https://stackoverflow.com/questions/26126442/combining-text-stemming-and-removal-of-punctuation-in-nltk-and-scikit-learn
# based on http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(texto):
    tokens = nltk.word_tokenize(texto)
    tokens = [i for i in tokens if i not in string.punctuation]
    stems = stem_tokens(tokens, stemmer)
    return stems

In [18]:
corpus_sem_stopwors_tokenizada= list(map(tokenize, corpus_sem_stopwords))

# 3) Lematização com a Lib stanza

In [ ]:
def download_stanza_portugues():
    """
    Faz o download do stanza em portugues
    """
    stanza.download(lang='pt')

In [ ]:
# Só é necessário executar o download uma vez
#download_stanza_portugues()

In [ ]:
def tokenizer_and_lemmatizer(text):
    """
        Performs tokenization and lemmatization on input text

    Args:
        text: A string with the content of the text

    Returns:
        A stanza Document with the tokens and lemmas

    """
    nlp = stanza.Pipeline('pt', processors='tokenize,mwt,pos,lemma')
    return nlp(text)

In [ ]:
def show_nlp_doc(doc):
    """
    Imprime os tokens (somente para debug)
    """
    sentence_id = 0
    for sentence in doc.sentences:
        sentence_id += 1
        print('\nSentença {}:'.format(sentence_id))
        for word in sentence.words:
            print('palavra = {}, lema = {}, id = {}'.format(word.text, word.lemma, word.id))


In [ ]:
#directory_path = r'C:\Users\r211315\Documents\ia_express\arquivos'
#directory_path = r'/home/vagnersv/my_tensorflow/arquivos'

In [ ]:
import stanza

txt3_processed = tokenizer_and_lemmatizer(corpus[0])

In [ ]:
show_nlp_doc(txt3_processed)

# 4) Lematização manual com inspiração no trabalho descrito no [link](https://github.com/rikarudo/LemPORT) (Atividade desafio)

# 5) Implementar API para determinar as seguintes informações do resultados obtidos em 3 e/ou 4 :

# 5.1) Term Frequency (TF):
𝑇𝐹 = 𝑞𝑡𝑑 𝑑𝑒 𝑜𝑐𝑜𝑟𝑟ê𝑛𝑐𝑖𝑎 𝑑𝑜 𝑡𝑒𝑟𝑚𝑜 𝑒𝑚 𝑢𝑚 𝑡𝑒𝑥𝑡𝑜 / 𝑞𝑢𝑎𝑛𝑡𝑖𝑑𝑎𝑑𝑒 𝑡𝑜𝑡𝑎𝑙 𝑑𝑒 𝑝𝑎𝑙𝑎𝑣𝑟𝑎𝑠 𝑑𝑜 𝑡𝑒𝑥𝑡𝑜

Referência:
[(Calculate TF-IDF in NLP (Simple Example)](https://youtu.be/vZAXpvHhQow)

[scikit-exemplos](https://dadosaocubo.com/nlp-com-scikit-learn/)

[Turing](https://github.com/turing-usp/BoW-e-TFIDF/blob/master/BoW_e_TFIDF.ipynb)

https://www.computersciencemaster.com.br/como-implementar-o-tf-idf-em-python/

https://medium.com/analytics-vidhya/demonstrating-calculation-of-tf-idf-from-sklearn-4f9526e7e78b

In [20]:
def map_occurrences(string_list):
    string_count = {}
    for item in string_list:
        if item not in string_count:
            count = string_list.count(item)
            string_count[item] = count
    return string_count

In [21]:
ocorrencias_termo= list(map(map_occurrences, corpus_sem_stopwors_tokenizada))

In [22]:
ocorrencias_termo

[{'tartaruga': 1,
  'lenta': 1,
  'pula': 1,
  'sobr': 1,
  'raposa': 1,
  'preguiçosa': 1,
  'rapasa': 1,
  'baralho': 1},
 {'rápida': 1,
  'raposa': 2,
  'marrom': 1,
  'pula': 1,
  'sobr': 1,
  'cão': 1,
  'preguiçoso': 1,
  'marota': 1}]

In [23]:
termos_documento= list(map(len, corpus_sem_stopwors_tokenizada))

In [24]:
termos_documento

[8, 9]

In [25]:
result={}
def calculaFT(ocorrencias_termo, termos_documento):
    for i in range(len(ocorrencias_termo)):
        result[i]= dict(map(lambda kv: (kv, ocorrencias_termo[i][kv]/termos_documento[i]), ocorrencias_termo[i]))
    return result

In [26]:
frequencia_termo= calculaFT(ocorrencias_termo, termos_documento)

In [27]:
frequencia_termo

{0: {'tartaruga': 0.125,
  'lenta': 0.125,
  'pula': 0.125,
  'sobr': 0.125,
  'raposa': 0.125,
  'preguiçosa': 0.125,
  'rapasa': 0.125,
  'baralho': 0.125},
 1: {'rápida': 0.1111111111111111,
  'raposa': 0.2222222222222222,
  'marrom': 0.1111111111111111,
  'pula': 0.1111111111111111,
  'sobr': 0.1111111111111111,
  'cão': 0.1111111111111111,
  'preguiçoso': 0.1111111111111111,
  'marota': 0.1111111111111111}}

In [28]:
import pandas as pd
frequencia_termo_df = pd.concat({k: pd.Series(v) for k, v in frequencia_termo.items()}).reset_index()
frequencia_termo_df.columns = ['corpus', 'palavra', 'TF ']

In [29]:
frequencia_termo_df

,corpus,palavra,TF
0,0,tartaruga,0.125000
1,0,lenta,0.125000
2,0,pula,0.125000
3,0,sobr,0.125000
4,0,raposa,0.125000
5,0,preguiçosa,0.125000
6,0,rapasa,0.125000
7,0,baralho,0.125000
8,1,rápida,0.111111
9,1,raposa,0.222222


# 5.2) Document Frequency (DF)
𝐷𝐹 = 𝑞𝑡𝑑 𝑑𝑒 𝑜𝑐𝑜𝑟𝑟ê𝑛𝑐𝑖𝑎 𝑑𝑜 𝑡𝑒𝑟𝑚𝑜 𝑒𝑚 𝑢𝑚 𝑐𝑜𝑛𝑗𝑢𝑛𝑡𝑜 𝑑𝑒 𝑑𝑜𝑐𝑢𝑚𝑒𝑛𝑡𝑜𝑠    
https://kavita-ganesan.com/what-is-document-frequency/#.YagKOCVv894

In [30]:
todas_palavras= []
for i in range (len(ocorrencias_termo)):
    for item in list(ocorrencias_termo[i].keys()):
        if item not in todas_palavras:
            todas_palavras.append(item)

In [32]:
numero_palavras_d1 = dict.fromkeys(todas_palavras, 0)
for word in corpus_sem_stopwors_tokenizada[0]:
        numero_palavras_d1[word] += 1

In [34]:
numero_palavras_d2 = dict.fromkeys(todas_palavras, 0)
for word in corpus_sem_stopwors_tokenizada[1]:
    numero_palavras_d2[word] += 1

In [35]:
documentos = [numero_palavras_d1, numero_palavras_d2]
document_frequency = dict.fromkeys(numero_palavras_d1.keys(), 0)
for document in documentos:
    for word, val in document.items():
        if val > 0:
            document_frequency[word] += 1

In [37]:
document_frequency

{'tartaruga': 1,
 'lenta': 1,
 'pula': 2,
 'sobr': 2,
 'raposa': 2,
 'preguiçosa': 1,
 'rapasa': 1,
 'baralho': 1,
 'rápida': 1,
 'marrom': 1,
 'cão': 1,
 'preguiçoso': 1,
 'marota': 1}

In [ ]:
pd.DataFrame.from_dict(document_frequency, orient='index', columns=['DF'])

# 5.3) Inverse Document Frequency (IDF)
𝐼𝐷𝐹 = 𝑙𝑜𝑔(𝑞𝑡𝑑 𝑑𝑒 𝑑𝑜𝑐𝑢𝑚𝑒𝑛𝑡𝑜𝑠 / (𝐷𝐹 + 1))

Referências:           
[stackoverflow](https://stackoverflow.com/questions/48431173/is-there-a-way-to-get-only-the-idf-values-of-words-using-scikit-or-any-other-pyt)     
[scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [38]:
import math

idf= {k: math.log10((len(corpus)/v)) for k, v in document_frequency.items()}

In [39]:
idf

{'tartaruga': 0.3010299956639812,
 'lenta': 0.3010299956639812,
 'pula': 0.0,
 'sobr': 0.0,
 'raposa': 0.0,
 'preguiçosa': 0.3010299956639812,
 'rapasa': 0.3010299956639812,
 'baralho': 0.3010299956639812,
 'rápida': 0.3010299956639812,
 'marrom': 0.3010299956639812,
 'cão': 0.3010299956639812,
 'preguiçoso': 0.3010299956639812,
 'marota': 0.3010299956639812}

In [40]:
document_frequency_mais_um = {k: v+1 for k, v in document_frequency.items()}

In [41]:
idf_ = {k: math.log10((len(corpus)/v)) for k, v in document_frequency_mais_um.items()}

In [42]:
idf_

{'tartaruga': 0.0,
 'lenta': 0.0,
 'pula': -0.17609125905568127,
 'sobr': -0.17609125905568127,
 'raposa': -0.17609125905568127,
 'preguiçosa': 0.0,
 'rapasa': 0.0,
 'baralho': 0.0,
 'rápida': 0.0,
 'marrom': 0.0,
 'cão': 0.0,
 'preguiçoso': 0.0,
 'marota': 0.0}

# 5.4) TF-IDF
𝑇𝐹 − 𝐼𝐷𝐹 = 𝐼𝐷𝐹 * 𝑇𝐹

In [43]:
frequencia_termo

{0: {'tartaruga': 0.125,
  'lenta': 0.125,
  'pula': 0.125,
  'sobr': 0.125,
  'raposa': 0.125,
  'preguiçosa': 0.125,
  'rapasa': 0.125,
  'baralho': 0.125},
 1: {'rápida': 0.1111111111111111,
  'raposa': 0.2222222222222222,
  'marrom': 0.1111111111111111,
  'pula': 0.1111111111111111,
  'sobr': 0.1111111111111111,
  'cão': 0.1111111111111111,
  'preguiçoso': 0.1111111111111111,
  'marota': 0.1111111111111111}}

In [44]:
idf

{'tartaruga': 0.3010299956639812,
 'lenta': 0.3010299956639812,
 'pula': 0.0,
 'sobr': 0.0,
 'raposa': 0.0,
 'preguiçosa': 0.3010299956639812,
 'rapasa': 0.3010299956639812,
 'baralho': 0.3010299956639812,
 'rápida': 0.3010299956639812,
 'marrom': 0.3010299956639812,
 'cão': 0.3010299956639812,
 'preguiçoso': 0.3010299956639812,
 'marota': 0.3010299956639812}

In [ ]:
def retorna_idf(palavra):
    for chave, valor in idf.items():
         if chave == palavra:
             return valor

In [ ]:
retorna_idf('rápida')

In [ ]:
for i in range(len(ocorrencias_termo)):
    for k,v in frequencia_termo[0].items():
        print(i)
        print(k, v)
        print(retorna_idf(k))

In [ ]:
for i in range(len(corpus)):
    

In [ ]:
a

In [ ]:
print(list(TF_ID))

In [ ]:
teste = dict(map(lambda kv: (kv, frequencia_termo[0][kv]/idf), frequencia_termo[0]))

In [ ]:
retorna_idf('lenta')

In [ ]:
frequencia_termo_

In [ ]:
for k,v in frequencia_termo.items():
    print(k, "-", v)

In [ ]:
idf= {k: math.log10((len(corpus)/v)) for k, v in document_frequency.items()}

# 5.5) Lista de strings com proximidade até 2 dos 5 termos de maior TF-IDF. Essas strings devem ser acompanhadas de seu valor de TF.

# 6) Gerar um arquivo csv que possui todas as palavras de todos os documentos na primeira coluna, em que cada linha é um token. Para cada token, informe nas colunas vizinhas as informações determinadas no objetivo 5.

# 7) Gerar nuvem de palavras para análise visual tal como exemplo abaixo. Cada ponto central será um dos 5 termos de maior TF-IDF. As conexões são as palavras próximas obtidas em 5.4. O tamanho do círculo da palavra é baseado no TF dela. O maior círculo que conecta o termo central será normalizado para palavras de maior TF do conjunto (desafio)

# Tópicos de Auxílio
Se realizada a lematização manual, os resultados seguintes são duplicados e a comparação será realizada analisando a nuvem de palavras de cada implementação.
Informações sobre as métricas utilizadas
https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-worlddataset-
796d339a4089
Atividade desafio de determinação da nuvem de palavras
https://www.kaggle.com/arthurtok/ghastly-network-and-d3-js-force-directed-graphs

http://andrewtrick.com/stormlight_network.html

https://github.com/corymaklin/tfidf/blob/master/tfidf.ipynb

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

my_data = ["hello how are you", "hello who are you", "i am not you"]

tf = TfidfVectorizer(use_idf=True)
tf.fit_transform(my_data)

idf = tf.idf_ 

#[BONUS] if you want to get the idf value for a particular word:

# If you want to get the idf value for a particular word, here "hello"    
tf.idf_[tf.vocabulary_["hello"]]


In [ ]:
len(my_data)

In [ ]:
my_data[0]